# Задание №1

Импортируем библиотеку для кодирования и декодирования строк. Является стандартной библиотекой python3

In [91]:
import codecs


Входная строка

In [92]:
hexString="49276d207374756479696e672043727970746f677261706879206c696b6520436c6175646520456c776f6f64205368616e6e6f6e21"

Сначала декодируем HEX и получаем закодированное сообщение на английском языке

In [93]:
decodedHex = codecs.decode(hexString,'hex')
print(decodedHex)

b"I'm studying Cryptography like Claude Elwood Shannon!"


Затем необходимо закодировать наше сообщение в base64, в итоге получим зашифрованное сообщение в base64

In [94]:
encodedBase64=codecs.encode(decodedHex,'base64')
print(encodedBase64)

b'SSdtIHN0dWR5aW5nIENyeXB0b2dyYXBoeSBsaWtlIENsYXVkZSBFbHdvb2QgU2hhbm5vbiE=\n'


Остается лишь избавиться от служебных символов и получить итоговое сообщение


In [95]:
decodedBase64 = encodedBase64.decode()
print(decodedBase64)

SSdtIHN0dWR5aW5nIENyeXB0b2dyYXBoeSBsaWtlIENsYXVkZSBFbHdvb2QgU2hhbm5vbiE=



# Задание №2

Сначала напишем функцию, вычисляющую XOR для двух строк одинаковой длины. Для этого нужно посимвольно произвести операцию XOR над каждой строкой.

In [96]:
def xorForTwoHexString(a, b):
    result = [] # Результирующий массив, куда будем записывать результат XOR
    zippedStrings = zip(a, b) # Массив с кортежами из символов входных строк
    for (x,y) in zippedStrings:
      xor = int(x,16) ^ int(y,16) # Каждый символ переводим в число в 16-ой системе и делаем XOR
      hexXor = hex(xor).split('x')[-1] # Переводим в hex
      result.append(hexXor)
    return "".join(result) # Соединяем все в общую строку

Добавляем входные данные

In [97]:
s1 = "506561636520416c6c204f7665722054686520576f726c64"
s2 = "4949544353551c0111001f010100061a021f010100061a02"

Выводим результат

In [98]:
print(xorForTwoHexString(s1, s2))

192c352036755d6d7d2050776472264e6a7a21566f747666


# Задание №3

Входные данные

In [99]:
inputText = '19367831362e3d2b2c353d362c783136783336372f343d3c3f3d7839342f39212b782839212b782c303d783a3d2b2c7831362c3d2a3d2b2c'

Так как символ при кодировке имеет длину 2, то распилим наш исходный текст на отрезки длинной 2. Также посчитаем для каждого частоту вхождения

In [100]:
pairCounts = {}
for i in range(0, len(inputText), 2):
    currentPair = inputText[i:i+2]
    if currentPair in pairCounts.keys():
        pairCounts[currentPair] += 1
    else:
        pairCounts[currentPair] = 1

Найдем самые часто встречающиеся пары. Это и будет наш ключ "заксоренный" с самым часто встречающимся символом

In [101]:
maxCount = max(pairCounts.values())
keys = []
pairKeys = list(pairCounts.keys())
pairValues = list(pairCounts.values())
for i in range(0, len(pairKeys)):
    if pairValues[i] == maxCount:
        keys.append(pairKeys[i])

Дальше, нам нужно привести ключ в исходное состояние, для этого выполним для каждого ключа операцию XOR с самой популярной буквой английского алфавита (в интернете говорят что это буква "е"). Также напишем две вспомогательные функции xor и xorWithMustPopularLetter

In [102]:
def xor(str1, str2):
    return int(str1, 16) ^ int(str2, 16)

def xorWithMustPopularLetter(str2):
    return hex(xor('e'.encode('ascii').hex(), str2))

for i in range(0, len(keys)):
    keys[i] = xorWithMustPopularLetter(keys[i])

Затем переведем нашу исходную строку в пары символов

In [103]:
inputText = list(map(lambda i: inputText[i:i+2], range(0, len(inputText), 2)))

Далее расшифруем наше сообщение с помощью полученных ранее ключей. Также при помощзи операции XOR

In [104]:
decodeMessages = []
for key in keys:
    resultMessage = []
    for letter in inputText:
      resultMessage.append(chr(xor(key, letter)))
    decodeMessages.append(resultMessage)

Выводим результат.В итоге из 2 ключей очевидно подошел второй.



In [105]:
for message in decodeMessages:
    print(''.join(message))

+e,+3 61( +1e,+e.+*2) !" e$)2$<6e5$<6e1- e' 61e,+1 7 61
An investment in knowledge always pays the best interest


# Задание №4

Для начала прочитаем исходные файлы

In [106]:
with open('task140.input') as f:
    file1Lines = f.readlines()

with open('task141.input') as f:
    file2Lines = f.readlines()

with open('task142.input') as f:
    file3Lines = f.readlines()

Также прочитаем сгенерированный на [этом](https://ru.lipsum.com) сайте файл, содержащий текст (Lorem ipsum) для того, чтобы использовать его символы для поиска часто встречающихся символов

In [107]:
with open('lorem-ipsum.input') as f:
    loremText = f.read()

Далее напишем вспомогательные функции:

*   перевод из 16-ой в 2-ую систему
*   XOR над двумя бинарными строками
*   счетчик частоты вхождения символов, возвращает отсортированный по частоте словарь



In [108]:
def fromHexToBin(str16):
    return "{0:08b}".format(int(str16, 16))

def binaryXor(str1, str2):
    return int(str1, 2) ^ int(str2, 2)

def frequencyCounter(string, step):
    pairCounts = {}
    for i in range(0, len(string), step):
        currentPair = string[i:i+step]
        if currentPair in pairCounts.keys():
            pairCounts[currentPair] += 1
        else:
            pairCounts[currentPair] = 1
    return dict(sorted(pairCounts.items(), key=lambda item: item[1],reverse=True))

Затем расчитаем частоту вхождения символов для нашей "Lorem Ipsum"

In [109]:
loremFrequency = frequencyCounter(loremText,1)

Дальше напишем функцию для расшифровки слов. Она работает по аналогии с программой из 3-его задания за одним исключением: там как "самый частый символ" я взял букву "е". Здесь же идет перебор по символам, взятым из "Lorem Ipsum". Для определения "подходимости" строки используем регулярное выражение, проверяющее что расшифрованная строка содержит только буквы, цифры и пробелы. Для этого импортируем библиотеку re

In [110]:
import re

def decodeText(text):
    textFrequency = frequencyCounter(text,2)
    for loremChar in loremFrequency.keys():
        for textChar in textFrequency.keys():
            loremBinChar = bin(int(ord(loremChar)))
            textBinChar = fromHexToBin(textChar)
            xorBinKey = bin(binaryXor(loremBinChar, textBinChar))
            decodedText = []
            for i in range(0, len(text), 2):
                binTextForDecode = fromHexToBin(text[i:i + 2])
                xorDecodedChar = chr(binaryXor(xorBinKey, binTextForDecode))
                decodedText.append(xorDecodedChar)
            decodedText = ''.join(decodedText)
                        
            if re.findall('[\s\w\d]*', decodedText)[0] == decodedText:
                return decodedText
    return None

Далее проходимся по каждой строке наших файлов, для нахождения закодированной строки. После того как находим ее, прекращаем обработку других строк, так как по условию задачи в каждом файле лишь одна закодированная строка

In [111]:
for num, row in enumerate(file1Lines):
    res = decodeText(row.strip())
    if res is not None:
        print(num, res)
        break
for num, row in enumerate(file2Lines):
    res = decodeText(row.strip())
    if res is not None:
        print(num, res)
        break
for num, row in enumerate(file3Lines):
    res = decodeText(row.strip())
    if res is not None:
        print(num, res)
        break

373 Kakim umnym ja sebja chuvstvuu
170 Now that the party is jumping

373 Kakim umnym ja sebja chuvstvuu


# Задание №5

Для начала прочитаем исходные файлы

In [112]:
with open('task15.input') as f:
    fileText = f.read()

Создадим ключ

In [113]:
secretKey = 'Shannon'

Возьмем также функцию из предыдущего задания

In [114]:
def binaryXor(str1, str2):
    return int(str1, 2) ^ int(str2, 2)

Далее пройдемся по каждому символу входного текста и применим к нему операцию XOR. Каждый символ сделаем бинарным. Также результат переведем в hex

In [115]:
encodedResult = []
for i in range(0, len(fileText)):
    textBinChar = bin(int(ord(fileText[i])))
    secretKeyBinChar = bin(int(ord(secretKey[i % len(secretKey)])))
    encodedResult.append(binaryXor(textBinChar, secretKeyBinChar))
encodedResult = bytes(encodedResult).hex()


Выведем результат

In [116]:
print(encodedResult)

00000000000000730b0e001a1d07311d150b0a4f1a3c4815060b4f083a0d0d0a4e0008730b13171e1b0f3d090d171d061d730e0e1c4e010f27010e000f034e370d070b001c0b730c141c070109733f0e1c020b4e0409134e272642736208000d031b37010f094e07072048030f1d060d731f0e1c054f013d4802010a0a0c210d000507010973090f0a4e1c0b301d130b4e1b0b3f0d020103021b3d01020f1a06013d1b4f64


# Задание № 6

Для начала прочитаем исходные файлы

In [117]:
with open('task160.input') as f:
    file1Text = f.read().strip()

with open('task161.input') as f:
    file2Text = f.read().strip()

Исходные файлы закодированы в Base64. Переведем их в HEX

In [118]:
import base64
file1TextHex = base64.b64decode(file1Text).hex()
file2TextHex = base64.b64decode(file2Text).hex()

Дальше переведем HEX в обычные строки

In [119]:
file1TextString = bytearray.fromhex(file1TextHex).decode("latin-1")
file2TextString = bytearray.fromhex(file2TextHex).decode("latin-1")

Также возьмем наш "Lorem Ipsum" из предыдущих заданий, для составления "самых частых символов"

In [120]:
with open('lorem-ipsum.input') as f:
    loremText = f.read().strip()

И возьмем функцию, для нахождения частоты вхождения символа в строку

In [121]:
def frequencyCounter(string, step):
    pairCounts = {}
    for i in range(0, len(string), step):
        currentPair = string[i:i+step]
        if currentPair in pairCounts.keys():
            pairCounts[currentPair] += 1
        else:
            pairCounts[currentPair] = 1
    return dict(sorted(pairCounts.items(), key=lambda item: item[1],reverse=True))

Для проверки ключа на "пригодность" будем использовать собственный валидатор, основанный на библиотеке string

In [122]:
import string
validCharacters = string.ascii_uppercase + string.ascii_lowercase + string.digits + '.,!?;:\'- \n'
def validateText(text):
    return all(c in validCharacters for c in text)

Так как длина ключа заранее нам не известна, необходимо вычислять ее для каждого текста. Вычислять будем смещением текста и поиском наибольшего "процента" повторений

In [123]:
def calculateKeyLength(textData):
    maxFrequencyPercent = 0
    maxLength = 0
    for i in range(int(len(textData)/2)):
        offsetText = textData[i+1:] + textData[:i+1]
        xorText = xorStringWithKey(textData, offsetText)
        xorTextFrequency = frequencyCounter(xorText,1)
        currentFrequencyPercent = list(xorTextFrequency.values())[0]/len(textData)
        if currentFrequencyPercent > maxFrequencyPercent:
            maxFrequencyPercent = currentFrequencyPercent
            maxLength = i+1  
            
    return maxLength

Для произведения операции XOR с текстом и ключом напишем следующую функцию. Для "зацикливания" ключа используем утилиту cycle из библиотеки itertools

In [124]:
from itertools import cycle

def xorStringWithKey(text, key):
    return ''.join(chr(ord(x) ^ ord(y)) for (x,y) in zip(text, cycle(key)))

Также напишем функцию для нахождения ключа. Здесь мы и будем использовать "Lorem Ipsum"



In [125]:
loremFrequency = frequencyCounter(loremText,1)

def findKey(text):
    textFrequency = frequencyCounter(text,1)
    possibleKeys = []
    for textChar in textFrequency.keys():
        for loremChar in loremFrequency.keys():
            possibleKeys.append(chr(ord(textChar) ^ ord(loremChar)))
    resultKey = ""
    for key in possibleKeys:
        resultKey = xorStringWithKey(text, key)
        if validateText(resultKey):
            resultKey = key
            break
    else:
        return
    
    return resultKey

Далее самая главная функция по декодированию текста. На основе длины ключа решаем сколько групп у нас будет.

In [127]:
def decodeText(text):
    keyLength = calculateKeyLength(text)
    groups = {}
    for i, char in enumerate(text):
        if i % keyLength in groups:
            groups[i % keyLength] += char
        else:
            groups[i % keyLength] = char

    keyChars = []
    for i, groupText in groups.items():
        keyChars.append(findKey(groupText))
    key = ''.join(keyChars)
    decodedText = xorStringWithKey(text, key);
    print(f'Key: {key}')
    print(f'Decoded text: {decodedText}')
            
decodeText(file1TextString)
print('-------------------------------------------------------------------')
decodeText(file2TextString)

Key: Robert Laurence Binyon
Decoded text: How dark, how quiet sleeps the vale below!
In the dim farms, look, not a window shines:
Distantly heard among the lonely pines,
How soft the languid autumn breezes flow
Past me, and kiss my hair, and cheek, and mouth!
Half--veiled is the calm sky:
Jupiter's kingly eye
Alone glows full in the unclouded South.

Alas! and can sweet Night avail to heal
Not one of the world's wounds? Must I, even here,
Still listen with the mind's too wakeful ear
To that sad sound, which in my flesh I feel;
Sound of unresting, unrejoicing feet,
With feverish steps or slow
For ever, to and fro,
Pacing the gay, thronged, friendless, stony street?

Nature is free; but Man the eternal slave
Of care and passion. Must I deem that true?
With fields and quiet have we nought to do,
Because our spirits for ever crave and crave,
And never found their satisfaction yet?
World, is thy heart so cold,
So deeply weary and old,
That thy sole business is but to forget?

No, no! these 